In [65]:
#model definition

import random
import time
from collections import deque
import numpy as np

import torch as tc
import torch.nn as nn
import torch.optim as opt
import torch.nn.init as init
DEVICE = tc.device("cuda" if tc.cuda.is_available() else "cpu")

import gym
import gym_snake
env = gym.make('snake-v0')

def obs2input(obs):
    snake = obs["snake"]
    dy,dx = obs["dot"]
    
    arr=[[[0.0 for _ in range(env.height)] for __ in range(env.width)] for ___ in range(2)]
    
    for i in range(len(snake)):
        y,x=snake[i]
        arr[0][y][x]=1.0+i/len(snake)
    arr[1][dy][dx]=1.0
    
    return arr

class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.in_ch=2
        self.out_ch=4
        
        self.c1=nn.Sequential(
#             nn.Conv2d(self.in_ch,10,3,padding=1,padding_mode="circular"),
#             nn.Dropout2d(0.3),
#             nn.LeakyReLU(0.1),
#             nn.Conv2d(10,15,3,padding=1,padding_mode="circular"),
#             nn.Dropout2d(0.3),
#             nn.LeakyReLU(0.1),
#             nn.Conv2d(15,8,3,padding=1,padding_mode="circular"),
#             nn.Dropout2d(0.3),
#             nn.LeakyReLU(0.1),
#             nn.Conv2d(8,self.out_ch,3,padding=1,padding_mode="circular"),
#             nn.Dropout2d(0.3),
#             nn.LeakyReLU(0.1),
        )
        
        self.d1=nn.Sequential(
            #nn.Linear(self.out_ch*env.height*env.width,256),
            nn.Linear(self.in_ch*env.height*env.width,256),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.1),
            
            nn.Linear(256,128),
            nn.Dropout(0.35),
            nn.LeakyReLU(0.1),
            
            nn.Linear(128,4),
        )
        for m in [self.c1,self.d1]:
            if type(m)==nn.Linear or type(m)==nn.Conv2d:
                init.xavier_uniform_(m.weight.data)
                init.xavier_uniform_(m.bias.data)
                
    def forward(self,x):
        x=x.to(DEVICE).reshape((-1,self.in_ch,env.height,env.width))
        #x=self.c1(x)
        x=x.reshape(-1,self.in_ch*env.height*env.width)
        return self.d1(x)

In [196]:
#load
net_trained=NN().cuda()
net_trained.load_state_dict(tc.load('./netw.pt'))

<All keys matched successfully>

In [252]:
net_trained.eval()

obs=env.reset()
while True:
    res=net_trained(tc.tensor(obs2input(obs)).to(DEVICE)).detach().squeeze().tolist()
    obs, rwd, done, _ = env.step(res.index(max(res)))
    env.render()
    if done:
        break
env.close()